# Document Clustering and Topic Modeling 

## Contents

* [Part 1: Load Data](#Part-1:-Load-Data)
* [Part 2: Tokenizing and Stemming](#Part-2:-Tokenizing-and-Stemming)
* [Part 3: TF-IDF](#Part-3:-TF-IDF)
* [Part 4: K-means clustering](#Part-4:-K-means-clustering)
* [Part 5: Topic Modeling - Latent Dirichlet Allocation](#Part-5:-Topic-Modeling---Latent-Dirichlet-Allocation)


In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file = drive.CreateFile({'id':'1UIS6YrBoeIrg6i4qEhXrF-NEK0L3W42_'}) 
file.GetContentFile('data.tsv')  

# Part 1: Load Data

In [ ]:
import numpy as np
import pandas as pd
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv('data.tsv', sep='\t', error_bad_lines=False) #！！

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkip

In [ ]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [ ]:
# Remove missing value
df.dropna(subset=['review_body'],inplace=True)

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960056 entries, 0 to 960055
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960056 non-null  object
 1   customer_id        960056 non-null  int64 
 2   review_id          960056 non-null  object
 3   product_id         960056 non-null  object
 4   product_parent     960056 non-null  int64 
 5   product_title      960054 non-null  object
 6   product_category   960056 non-null  object
 7   star_rating        960056 non-null  int64 
 8   helpful_votes      960056 non-null  int64 
 9   total_votes        960056 non-null  int64 
 10  vine               960056 non-null  object
 11  verified_purchase  960056 non-null  object
 12  review_headline    960049 non-null  object
 13  review_body        960056 non-null  object
 14  review_date        960052 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [ ]:
# training data
data = df.loc[:1999, 'review_body'].tolist()

# Part 2: Tokenizing and Stemming

In [ ]:
# Load stopwords and stemmer function from NLTK library.

# Stemming is the process of breaking a word down into its root.
# Stop words are words like "a", "the", or "in" which don't convey significant meaning.

# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english') 
stopwords.append("'s") 
stopwords.append("'m")
stopwords.append("br") 
stopwords.append("watch") 

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 183 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [ ]:
# Use our defined functions to analyze (i.e. tokenize, stem) our reviews.
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 
stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text): 
    tokens = []   
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
     
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

# Part 3: TF-IDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  % sorted(inconsistent)


In total, there are 2000 reviews and 246 terms.


In [13]:
tfidf_matrix

<2000x246 sparse matrix of type '<class 'numpy.float64'>'
	with 14532 stored elements in Compressed Sparse Row format>

In [14]:
# Save the terms identified by TF-IDF. 
# words
tf_selected_words = tfidf_model.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
# print out words
tf_selected_words

['abl',
 'absolut',
 'accur',
 'actual',
 'add',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon',
 'anoth',
 'appear',
 'arriv',
 'attract',
 'automat',
 'awesom',
 'bad',
 'band',
 'batteri',
 'beauti',
 'best',
 'better',
 'big',
 'bit',
 'black',
 'blue',
 'bought',
 'box',
 'brand',
 'break',
 'bright',
 'broke',
 'broken',
 'button',
 'buy',
 'ca',
 'came',
 'case',
 'casio',
 'chang',
 'cheap',
 'check',
 'clasp',
 'classi',
 'clear',
 'clock',
 'collect',
 'color',
 'come',
 'comfort',
 'compliment',
 'cool',
 'cost',
 'coupl',
 'crown',
 'crystal',
 'cute',
 'dark',
 'date',
 'daughter',
 'day',
 'deal',
 'definit',
 'deliveri',
 'design',
 'dial',
 'differ',
 'digit',
 'disappoint',
 'display',
 'dress',
 'durabl',
 'easi',
 'easili',
 'eleg',
 'end',
 'everi',
 'everyday',
 'everyth',
 'exact',
 'excel',
 'expect',
 'expens',
 'extrem',
 'face',
 'fact',
 'far',
 'fast',
 'featur',
 'feel',
 'fell',
 'figur',
 'fine',
 'finish',
 'fit',
 'function',
 'gift',
 'g

# Part 4: K-means clustering

In [ ]:
# k-means clustering
from sklearn.cluster import KMeans

# set K
num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

## 4.1. Analyze K-means Result

In [17]:
# k-means clustering
from sklearn.cluster import KMeans
num_clusters = 10
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

# create DataFrame films from all of the input files.
product = { 'review': df[:2000].review_body, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [ ]:
frame.head(10)

,review,cluster
0,Absolutely love this watch! Get compliments al...,2
1,I love this watch it keeps time wonderfully.,2
2,Scratches,3
3,"It works well on me. However, I found cheaper ...",3
4,Beautiful watch face. The band looks nice all...,3
5,"i love this watch for my purpose, about the pe...",2
6,"for my wife and she loved it, looks great and ...",4
7,I was about to buy this thinking it was a Swis...,3
8,Watch is perfect. Rugged with the metal &#34;B...,4
9,Great quality and build.<br />The motors are r...,3


In [18]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
1,859
6,271
8,148
5,128
4,122
9,110
2,110
7,107
3,93
0,52


In [19]:
km.cluster_centers_

array([[0.        , 0.        , 0.        , ..., 0.        , 0.01220659,
        0.00641191],
       [0.00657573, 0.00186084, 0.00641657, ..., 0.01054739, 0.02378945,
        0.02100651],
       [0.00277647, 0.01823814, 0.00395462, ..., 0.00914958, 0.00288338,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03466263, 0.        , ..., 0.00600754, 0.00596094,
        0.        ],
       [0.        , 0.00493754, 0.00282315, ..., 0.        , 0.02867561,
        0.00776159]])

In [21]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews))
    print ()

<Document clustering result by K-means>
Cluster 0 words:excel,product,price,fast,good,qualiti,
Cluster 0 reviews (52 reviews): 
excelent product, Excellent product and seller very good service, Excelent for the price., I am extremely pleased with the product.  Excellent watch!, Very nice watch. I get compliments all the time and everyone asks where I got it. This is an excellent watch for business/formal types of gigs that doesn't break the bank. Not heavy on the wrist., Excellent Product, competitive price and quick delivery., Great item... excellent price, My husband loves it!  Excellent product!, excellent watch, Excellent quality. Solid. Would highly recommend., Excellent. My son loves it. He wants to swim with it, so we are yet to pass this test!, excellent, Excelent for the price., Excellent product!, Excelent..., Excellent quality and very fast shipping thank you., Excelent!, Excellent quality and very fast shipping thank you., Excellent product!...great deal!, Excellent feeling

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [22]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [23]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(2000, 5)
[[0.05980632 0.06081427 0.75834744 0.06057401 0.06045795]
 [0.08437639 0.08382157 0.6571357  0.0870328  0.08763354]
 [0.1000023  0.59867746 0.10100938 0.10000275 0.10030811]
 ...
 [0.06713775 0.08077364 0.06934641 0.06511992 0.71762227]
 [0.10145631 0.10011517 0.59729085 0.10042156 0.10071611]
 [0.2        0.2        0.2        0.2        0.2       ]]


In [24]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 246)
[[ 0.21096457  0.20164186  0.20118794 ...  0.20777257 20.44062984
   2.10727747]
 [ 6.26310212  0.21167343  0.27613424 ... 10.26791412  0.20204211
   0.28066742]
 [ 0.21790952 11.9871265   1.8068008  ...  0.20263932  7.15770791
   2.13272371]
 [ 0.20626269  0.20207949  3.83015448 ...  3.08791172  1.03337633
  19.98779841]
 [ 3.03120777  0.20310709  2.99350761 ...  0.20147352  6.03680166
   0.20218647]]


In [25]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.06,0.06,0.76,0.06,0.06,2
Doc1,0.08,0.08,0.66,0.09,0.09,2
Doc2,0.10,0.60,0.10,0.10,0.10,1
Doc3,0.46,0.06,0.06,0.36,0.06,0
Doc4,0.42,0.04,0.46,0.04,0.04,2
Doc5,0.07,0.69,0.08,0.08,0.08,1
Doc6,0.06,0.53,0.07,0.06,0.28,1
Doc7,0.06,0.06,0.06,0.75,0.06,3
Doc8,0.05,0.23,0.63,0.05,0.05,2
Doc9,0.06,0.77,0.06,0.06,0.06,1


In [26]:
df_document_topic['topic'].value_counts().to_frame()

,topic
2,583
0,408
3,403
1,315
4,291


In [29]:
print(lda.components_)
df_topic_words = pd.DataFrame(lda.components_)
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names
df_topic_words.head()

[[ 0.21096457  0.20164186  0.20118794 ...  0.20777257 20.44062984
   2.10727747]
 [ 6.26310212  0.21167343  0.27613424 ... 10.26791412  0.20204211
   0.28066742]
 [ 0.21790952 11.9871265   1.8068008  ...  0.20263932  7.15770791
   2.13272371]
 [ 0.20626269  0.20207949  3.83015448 ...  3.08791172  1.03337633
  19.98779841]
 [ 3.03120777  0.20310709  2.99350761 ...  0.20147352  6.03680166
   0.20218647]]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abl,absolut,accur,actual,add,adjust,alarm,alreadi,alway,amaz,amazon,anoth,appear,arriv,attract,automat,awesom,bad,band,batteri,beauti,best,better,big,bit,black,blue,bought,box,brand,break,bright,broke,broken,button,buy,ca,came,case,casio,...,someth,son,star,start,stop,strap,style,stylish,super,sure,surpris,tell,thank,thing,think,thought,time,timex,took,tri,turn,use,valu,want,watch,water,way,wear,week,weight,went,white,wife,wish,wo,work,worn,worth,wrist,year
Topic0,0.210965,0.201642,0.201188,0.203379,1.169422,6.090168,2.594985,0.204325,0.204867,0.200882,0.203170,1.596707,2.486418,1.322408,4.831283,0.202009,0.200647,3.903406,30.424509,0.201070,0.201555,0.203180,0.202914,18.551143,10.601682,0.925033,0.201699,2.014202,0.201407,0.203056,2.947116,0.204310,0.336373,0.202989,0.206207,2.177815,5.896478,1.189824,0.668545,4.322123,...,0.207799,0.265154,3.674408,0.200787,0.201474,7.159132,5.889066,0.200997,0.202492,0.716895,0.202295,0.202823,0.201481,1.208225,7.755355,0.204101,10.382485,0.599502,2.732699,0.202119,0.202042,10.293178,0.207850,1.756404,8.707821,0.390209,2.611187,2.546498,3.273494,6.702173,0.699479,0.200654,0.200861,6.298527,0.879901,10.547078,0.202331,0.207773,20.440630,2.107277
Topic1,6.263102,0.211673,0.276134,0.286323,0.201146,0.204643,0.200456,0.202156,4.670498,3.531796,0.201560,0.200611,0.830334,0.203374,0.201539,0.201525,20.748560,0.200978,11.464407,0.200879,0.201388,6.656886,9.185046,0.202590,0.202547,0.200475,0.200262,0.984306,1.984418,1.921831,0.201746,0.200106,15.199039,9.535978,0.200956,1.745614,0.201510,0.617946,4.900487,0.201179,...,2.211893,1.809422,2.695981,0.200572,0.200514,0.923653,1.823000,0.201562,0.201377,0.200726,0.204910,0.200482,0.200650,2.007680,0.846231,0.201176,2.891995,0.914434,0.201357,3.101904,0.200455,3.109201,1.841151,3.620262,6.749063,0.200828,0.201280,0.532070,0.468141,0.202240,0.209274,0.200364,0.410713,0.201579,0.201302,5.752360,0.202227,10.267914,0.202042,0.280667
Topic2,0.217910,11.987127,1.806801,8.385756,4.159015,3.781798,0.201032,2.221020,0.205244,14.703892,3.444535,3.070528,4.541019,0.203750,6.035702,1.306538,0.201855,0.204043,17.529951,0.201620,10.164604,1.628650,6.120011,12.279467,5.137439,11.238279,14.677446,20.033876,9.852664,2.198659,0.764387,3.451134,0.556571,0.204659,0.226995,8.830764,5.124122,7.183925,4.757161,0.201098,...,3.005944,1.687905,4.003264,4.901099,0.202465,18.410349,8.103950,13.342363,11.469815,4.182508,3.844094,11.980615,0.208143,4.914102,2.010122,2.936693,15.964308,3.410383,0.981541,0.201712,0.204790,3.268314,4.382789,7.400064,18.814344,0.203358,4.083907,13.867969,0.438769,1.998783,2.996126,6.113560,0.201305,0.204407,3.388351,0.631392,7.460992,0.202639,7.157708,2.132724
Topic3,0.206263,0.202079,3.830154,0.207953,0.204562,0.202572,10.223915,0.205300,0.656073,0.202875,4.338487,9.851128,0.203801,5.853256,0.203204,0.202234,0.201329,10.731539,0.215310,28.404704,0.201045,5.896130,0.204864,0.203470,0.202365,0.205652,0.201442,6.469833,0.204028,1.174262,2.983201,3.315639,5.150766,0.201906,9.682712,16.030426,2.763765,5.488627,0.883927,6.626445,...,2.575244,9.334230,0.206948,0.204491,15.261680,0.202463,0.201944,0.202020,0.201381,3.680113,3.077809,0.225818,0.578597,0.676596,3.105759,2.687804,24.888943,5.637814,2.287574,7.154194,4.096053,22.042298,2.724681,3.788778,3.978047,12.190632,0.202966,14.468352,13.716462,1.774796,0.953824,0.200768,0.200476,2.602304,0.205252,54.184576,0.201394,3.087912,1.033376,19.987798
Topic4,3.031208,0.203107,2.993508,2.566818,0.202829,2.927443,0.201669,6.089069,5.450258,0.202803,7.243979,0.202770,0.210535,8.731866,0.200755,6.590417,0.201259,2.450706,11.983412,0.201618,45.590694,0.200544,5.314122,0.293921,0.202130,0.200842,0.201755,3.877776,0.797591,3.150056,0.202136,0.203932,0.206079,0.201467,0.203283,4.980670,0.202177,3.238879,0.206440,0.200996,...,1.378268,2.631438,4.161258,4.413899,2.153830,0.200564,1.355393,0.200903,0.200438,1.502887,0.200940,0.201887,16.543214,6.593603,0.202858,2.526922,19.762953,1.611051,0.259382,2.403558,5.407313,2.865738,3.

In [30]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,good,look,band,face,wrist,size,fit,big,small,like,larg,link,invicta,littl,easi
Topic 1,nice,great,price,expect,look,awesom,love,broke,money,qualiti,simpl,deal,band,ok,everyth
Topic 2,love,like,gift,look,color,great,pretti,bought,watch,strap,band,feel,time,amaz,blue
Topic 3,work,excel,product,batteri,time,use,year,day,great,light,buy,stop,wear,week,replac
Topic 4,beauti,perfect,exact,time,set,wife,thank,look,love,wear,realli,want,item,band,pleas
